# W&B 시작하기

우리는 스프라이트(요정) 분류 모델 학습에 `wandb`를 추가하여 중요한 메트릭을 트래킹 및 시각화 할 수 있습니다.
이를 통해 모델 개선을 위해 유용한 정보를 습득할수도 있습니다.
또한 다른 실험들을 비교 분석하고 팀 멤버들과 협동하며, 결과물을 효과적으로 재생산할수도 있습니다.

In [ ]:
import math
from pathlib import Path
from types import SimpleNamespace
from tqdm.auto import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from utilities import get_dataloaders

import wandb

### 요정 분류

요정들을 분류하는 간단한 모델을 생성해봅시다. 아래 이미지에서 요정들과 요정 이미지를 볼 수 있을 것입니다.

In [ ]:
INPUT_SIZE = 3 * 16 * 16
OUTPUT_SIZE = 5
HIDDEN_SIZE = 256
NUM_WORKERS = 2
CLASSES = ["hero", "non-hero", "food", "spell", "side-facing"]
DATA_DIR = Path('./data/')
DEVICE = torch.device("cuda" if torch.cuda.is_available()  else "cpu")

def get_model(dropout):
    "Simple MLP with Dropout"
    return nn.Sequential(
        nn.Flatten(),
        nn.Linear(INPUT_SIZE, HIDDEN_SIZE),
        nn.BatchNorm1d(HIDDEN_SIZE),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(HIDDEN_SIZE, OUTPUT_SIZE)
    ).to(DEVICE)

In [ ]:
# 하이퍼파라미터를 저장하기 위한 config를 정의합니다.
config = SimpleNamespace(
    epochs = 2,
    batch_size = 128,
    lr = 1e-5,
    dropout = 0.5,
    slice_size = 10_000,
    valid_pct = 0.2,
)

위에서 정의한 config를 `wandb.init`에 매개변수로 전달합니다.

In [ ]:
def train_model(config):
    "Train a model with a given config"
    
    wandb.init(
        project="dlai_intro",
        config=config,
    )

    # 데이터를 가져옵니다
    train_dl, valid_dl = get_dataloaders(DATA_DIR, 
                                         config.batch_size, 
                                         config.slice_size, 
                                         config.valid_pct)
    n_steps_per_epoch = math.ceil(len(train_dl.dataset) / config.batch_size)

    # 간단한 MLP 모델
    model = get_model(config.dropout)

    # 손실 함수와 옵티마이저 설정
    loss_func = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=config.lr)

    example_ct = 0

    for epoch in tqdm(range(config.epochs), total=config.epochs):
        model.train()

        for step, (images, labels) in enumerate(train_dl):
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            outputs = model(images)
            train_loss = loss_func(outputs, labels)
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

            example_ct += len(images)
            metrics = {
                "train/train_loss": train_loss,
                "train/epoch": epoch + 1,
                "train/example_ct": example_ct
            }
            wandb.log(metrics)
            
        # validation 로스와 메트릭을 계산합니다
        val_loss, accuracy = validate_model(model, valid_dl, loss_func)
        val_metrics = {
            "val/val_loss": val_loss,
            "val/val_accuracy": accuracy
        }
        wandb.log(val_metrics)
    
    wandb.finish()

In [ ]:
def validate_model(model, valid_dl, loss_func):
    "Compute the performance of the model on the validation dataset"
    model.eval()
    val_loss = 0.0
    correct = 0

    with torch.inference_mode():
        for i, (images, labels) in enumerate(valid_dl):
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            # Forward pass
            outputs = model(images)
            val_loss += loss_func(outputs, labels) * labels.size(0)

            # Compute accuracy and accumulate
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            
    return val_loss / len(valid_dl.dataset), correct / len(valid_dl.dataset)

### W&B 계정

 https://wandb.ai/site 에서 [회원가입](https://wandb.ai/site) 하고 wandb 계정에 로그인합니다.  
 실험 결과와 다양한 W&B 기능을 이용할 수 있습니다.
 로그인하지 않고 익명 모드로도 W&B 기능을 이용할 수 있고, W&B 계정이 있다면 브라우저가 자동적으로 로그인 해줄 것입니다.

In [ ]:
wandb.login(anonymous="allow")

### 모델 학습

기본 config로 모델을 학습하고, W&B가 어떻게 작동하는지 살펴봅시다

In [ ]:
train_model(config)

In [ ]:
# 학습률을 변경하고 결과가 어떻게 달라지는지 확인헤보세요
config.lr = 1e-4
train_model(config)

In [ ]:
config.lr = 1e-4
train_model(config)

In [ ]:
config.dropout = 0.1
config.epochs = 1
train_model(config)

In [ ]:
config.lr = 1e-3
train_model(config)